## **Parte 2**
- Joaquín Campos - 22155
- Sofía García - 22210
- Julio García Salas - 22076

# Teoría
## 1) ¿Por qué un único paradigma es insuficiente? Diseño de un modelo híbrido para evaluar un nuevo metro

**Insuficiencia de un único paradigma**

- **Solo MBA (Modelado Basado en Agentes)**: Captura heterogeneidad (viajeros, automovilistas, operadores) y decisiones micro ($\text{ruta},\ \text{horario},\ \text{tolerancia al hacinamiento}$), pero **no** representa con precisión colas de servicio, capacidades y tiempos de espera gobernados por eventos discretos en estaciones y semáforos.
- **Solo DES (Eventos Discretos)**: Modela muy bien **servicios** y **recursos** (andenes, trenes, frecuencias, semáforos, carriles) con colas $M/M/s$ o variantes, pero **no** modela adaptaciones comportamentales (cambio de modo/horario/itinerario) ni spillovers entre individuos.
- **Solo DS (Dinámica de Sistemas)**: Ofrece visión macro de **flujos y stocks** (demanda pico/fuera de pico, congestión promedio, emisiones), pero **agrega** en exceso y pierde los efectos de composición/micro-incentivos.

**Arquitectura híbrida propuesta**

- **Capa MBA (micro) — Viajeros y Conductores**  
  - Agentes: viajeros con atributos $(\text{ingreso},\ \text{valor del tiempo},\ \text{elasticidad modal},\ \text{aversion al hacinamiento})$.  
  - Decisiones: elección de modo $(\text{auto},\ \text{bus},\ \text{metro})$, ruta y hora de salida mediante utilidad $U_{i,m} = -\alpha \cdot \text{tiempo}_{i,m} - \beta \cdot \text{costo}_{i,m} - \gamma \cdot \text{hacinamiento}_{m}$.
- **Capa DES (meso) — Operación de la red**  
  - **Metro**: procesos de llegada de trenes, embarque/desembarque, capacidad de tren/andén, control de intervalos (headway).  
  - **Vialidad/buses**: intersecciones, semáforos, carriles, colas, tiempos de servicio.  
  - Recursos y colas definen **tiempos de espera** y **rechazos por capacidad** en tiempo real.
- **Capa DS (macro) — Retroalimentaciones de sistema**  
  - Stocks/flows: $C(t)$ (congestión promedio), $D_m(t)$ (demanda por modo), $E(t)$ (emisiones), $S(t)$ (satisfacción).  
  - Ecuaciones de ejemplo:  
    - $\dfrac{dC}{dt} = \phi \cdot \text{flujo\_autos}(t) - \kappa \cdot \text{capacidad\_vial}$  
    - $\dfrac{dD_{\text{metro}}}{dt} = \eta \big(\text{utilidad\_metro}(t) - \overline{\text{utilidad}}\big)$  
    - $\dfrac{dE}{dt} = \sum_m \rho_m \cdot \text{veh\_km}_m(t)$

**Acoplamientos entre capas**

1. **MBA → DES**: Los agentes generan **llegadas** a estaciones/paradas y solicitan recursos (espacio en tren/bus, carril).  
2. **DES → MBA**: El DES devuelve **tiempos de espera/recorrido** y **rechazos**, que actualizan la utilidad $U_{i,m}$ y gatillan replanificación.  
3. **DES → DS**: Agregación periódica de **flujos**, **velocidades**, **rechazos**, **ocupaciones** para actualizar stocks/flows macro.  
4. **DS → MBA/DES**: El estado macro (p. ej., $C(t)$ o $S(t)$) **modula parámetros**: velocidades libres, propensión al cambio modal, frecuencia de metro en hora pico, campañas de información.

**Resultados**: El híbrido permite medir **impacto del metro** en **colas, tiempos, elección modal, congestión sistémica y emisiones**, capturando micro-decisión, operación y retroalimentaciones.

---

## 2) Patrón 1: MBA → DES — Transición fluida Agente→Entidad (AnyLogic o Mesa+SimPy)

**Concepto**: Un **viajero-agente (MBA)** llega a una **estación (DES)** y se convierte temporalmente en una **entidad en una cola/proceso** hasta completar el servicio (embarque–viaje–desembarque), para luego **restituir** su control como agente.

**Información a transferir (payload mínimo)**

- Identificación: $\text{id}$ único del agente.
- Estado de viaje: $\text{origen},\ \text{destino},\ \text{ruta},\ \text{itinerario}$, segmento actual.
- Parámetros de comportamiento relevantes para servicio: $\text{tolerancia\_espera},\ \text{valor\_tiempo},\ \text{aversion\_hacinamiento}$.
- Tiempo: timestamp de llegada, deadline (si aplica), ventana deseada.
- Metadatos para métricas: cohorte, motivo del viaje, ticket/tarifa, atributos socioeconómicos anonimizados.

**AnyLogic (idea técnica)**

1. En el **Agent** `Traveler`, al llegar a `Station`, llamar a `station.enterQueue(this)` y pasar un **object payload** (Java) con los campos anteriores.  
2. La **librería de Process Modeling** (DES) gestiona `Queue → Seize(trainCapacity) → Delay(rideTime) → Release`.  
3. En `onExit` del bloque final, invocar `payload.returnControl()` para **restaurar** el agente y continuar su plan (MBA).  
4. Registro de KPIs: `agent.trips.addLegStats(wait, inVehicle, deniedBoardings)`.

**Mesa (MBA) + SimPy (DES) (patrón abstracto)**

- **Estructura**:
  - Mesa ejecuta el **step** de agentes (decisión, movimiento macro).
  - SimPy corre en **tiempo continuo** para colas/recursos.

- **Handshake (pseudo)**:
  ```python
  # En el step() del agente (Mesa):
  if at_station:
      env.process(request_boarding(self.payload()))

  # Proceso DES (SimPy):
  def request_boarding(payload):
      arrival_t = env.now
      with train_capacity.request() as req:
          yield req
          wait = env.now - arrival_t
          ride_time = compute_ride_time(payload.route_segment)
          yield env.timeout(ride_time)
      # devolver control a MBA:
      mailbox.put({
          "id": payload["id"],
          "wait": wait,
          "ride_time": ride_time,
          "denied": False
      })

## Sincronización

- Un **mailbox/cola** comunica **eventos completados** hacia el modelo **Mesa**.  
- Cada **$k$** ticks de Mesa, se **drena** el mailbox para **actualizar** el estado de los agentes.  
- Si hay **rechazo por capacidad**, el proceso retorna `denied=True` y el **agente replantea su decisión** (MBA).

---

## 3) Patrón 3: DS dentro de un Agente — Escenario fuera de biología

### Escenario: Mercado minorista con firmas (agentes) que ajustan precio e inventario

**Agente (MBA):** Cada **firma** compite en un mercado local (heterogeneidad en costos, calidad, *branding*). Decide **precio**, **promoción** y **pedido a proveedor**. Interactúa con consumidores-agentes y competidores.

**DS interno (por firma):** Dinámica continua de **inventario** y **capital de marca**:

- Inventario $I(t)$ con entradas (pedidos recibidos) y salidas (ventas):  
  $\displaystyle \frac{dI}{dt} = \text{recepciones}(t) - \text{ventas}(t)$

- Capital de marca $B(t)$ que acumula con publicidad y decae por olvido/competencia:  
  $\displaystyle \frac{dB}{dt} = \lambda \cdot \text{ad\_spend}(t) - \mu \cdot B(t)$

- Demanda instantánea de la firma $q(t)$ depende de **precio relativo**, **stock** y **marca**:  
  $\displaystyle q(t) = \theta \cdot \frac{B(t)}{\text{precio}(t)} \cdot \sigma\!\big(\text{disponibilidad}(I(t))\big)$

**Acoplamiento multi-escala:**

- El DS interno actualiza $I(t)$ y $B(t)$ de cada firma; estos estados alteran su **atractividad** y **capacidad de servicio**.  
- A nivel MBA, consumidores eligen firma según utilidad que incluye $B(t)$ y **disponibilidad**; las ventas observadas **realimentan** el DS (consumo de inventario).  
- Permite estudiar **ciclos de *stockouts***, **guerras de precios** y **saturación publicitaria** emergentes.

---

## 4) Dos mayores desafíos al construir/validar un modelo híbrido y cómo abordarlos

### A) Calibración y coherencia temporal entre paradigmas

**Problema:** MBA opera por **ticks discretos**; DES por **eventos continuos**; DS por **integración** con paso $\Delta t$. El **desfase temporal** induce sesgos (p. ej., **esperas sub/sobreestimadas**) y **double counting** de demoras.

**Mitigaciones:**
- Definir un **reloj maestro** y **reglas de sincronización**:
  - Integrar DS con paso **$\Delta t$ fijo**; **procesar** la cola de eventos DES hasta $t+\Delta t$; luego **actualizar** decisiones MBA.
- **Sub-stepping adaptativo:** reducir $\Delta t$ cuando aumenta la carga de eventos (picos).
- **Calibración conjunta** con **objetivos multi-nivel**: ajustar parámetros para reproducir **distribuciones de espera** (DES), **shares modales** (MBA) e **indicadores macro** de **congestión/emisiones** (DS) simultáneamente.

### B) Complejidad de software y validación de resultados emergentes

**Problema:** El acoplamiento incrementa la **superficie de errores** (interfaces, *deadlocks*, desincronizaciones) y dificulta atribuir causas a **resultados emergentes** (identificabilidad).

**Mitigaciones:**
- **Contratos de interfaz** claros (payloads tipados, **unidades**, **convenciones de tiempo**) y **harness** de pruebas por capa (unitarias DES/MBA/DS + integración con escenarios canónicos).
- **Trazabilidad y telemetría:** *logs* de eventos con IDs, sondas de métricas en fronteras **MBA↔DES↔DS**, y **semillas** reproducibles.
- **Análisis de sensibilidad/SHAP global:** variar sistemáticamente parámetros clave y usar métodos de **screening** (Morris, Sobol) para aislar contribuciones.
- **Validación cruzada:**
  - **Face validity** con expertos de operación.  
  - **Comparación** contra series históricas: distribuciones de $\text{espera}$, $\text{viaje}$, $\text{rechazos}$ y **curvas AADT**.  
  - **Tests de Turing** entre versiones con paradigmas **separados** vs. **híbrido** para asegurar mejoras justificadas.
